In [ ]:
import pysm3
import pysm3.units as u
import numpy as np
import numpy.ma as ma
import healpy as hp
import pickle
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import pickle
from astropy.io import fits
import pandas as pd
from importlib import reload

from qubic import camb_interface as qc
import fgbuster as fgb

import warnings
warnings.filterwarnings("ignore")

import qubic
from qubic import NamasterLib as nam

center = qubic.equ2gal(0, -57)

rc('figure', figsize=(16, 7))
rc('font', size=15)


# Alm bias
When performing FGBuster-based component separation in alm space with partial sky, wee observe a systematic multiplicative bias on the recovered pixel numbers. Here we try to understand this effect.

We show that the effect comes from the $\ell$-space band-limited treatment performed by anafast. So for pure noise, which is wide-band, we have a significant effect, but this does not apply to signal which is band-limited... So hard to know what one should do with this...

But after all, this is not so clear that it eexplains what we see with FGBUster as this is noise-free simulations for now. So it should be signal only and therefore should not show this effect...

## 1) Full Sky - Noise Only
There is already a significant bias...

In [ ]:
nside = 256

# input map
mymap = np.random.randn(12*nside**2)

# Anafast
cl, alm = hp.anafast(mymap, alm=True)

# Synfast
mapback = hp.alm2map(alm, nside)

# Maps
hp.mollview(mymap, sub=(1,3,1), min=np.min(mymap), max=np.max(mymap)/10, title='In')
hp.mollview(mapback, sub=(1,3,2), min=np.min(mymap), max=np.max(mymap)/10, title='Out')
hp.mollview(mapback-mymap, sub=(1,3,3), title='Out - In')

# Scatter Plot
figure()
plot(mymap, mapback, 'o')
xlabel('In')
ylabel('Out')
myfit, covfit = np.polyfit(mymap, mapback, 1, cov=True)
xxx = np.linspace(np.min(mymap),np.max(mymap), 100)
plot(xxx, xxx, 'r--', label='y=x')
plot(xxx, np.poly1d(myfit)(xxx), lw=3, label='Slope: {0:5.2g} +/- {1:5.2e}'.format(myfit[0], np.sqrt(covfit[0,0])))
legend()


So here we do see a bias, but it is in the other direction w.r.t. what we find with FGBuster where the slope is 1.4 (also out/in). So there is a mystery here...

## 2) Partial Sky - Noise Only

In [ ]:
nside = 256
covmap = hp.ud_grade(hp.read_map('QUBIC-Typical-Coverage.fits'), nside)
thr = 0.1
mymask = (covmap > (np.max(covmap)*thr)).astype(int)
pixok = mymask > 0

# input map
mymap = np.random.randn(12*nside**2)
mymap[~pixok] = hp.UNSEEN

# Anafast
cl, alm = hp.anafast(mymap, alm=True)

# Synfast
mapback = hp.alm2map(alm, nside)
mapback[~pixok] = hp.UNSEEN

diffmap = mapback-mymap
diffmap[~pixok] = hp.UNSEEN

# Maps
hp.mollview(mymap, sub=(1,3,1), min=np.min(mymap[pixok]), max=np.max(mymap[pixok])/10, title='In')
hp.mollview(mapback, sub=(1,3,2), min=np.min(mymap[pixok]), max=np.max(mymap[pixok])/10, title='Out')
hp.mollview(diffmap, sub=(1,3,3), title='Out - In')

# Scatter Plot
figure()
plot(mymap[pixok], mapback[pixok], 'o')
xlabel('In')
ylabel('Out')
myfit, covfit = np.polyfit(mymap[pixok], mapback[pixok], 1, cov=True)
xxx = np.linspace(np.min(mymap[pixok]),np.max(mymap[pixok]), 100)
plot(xxx, xxx, 'r--', label='y=x')
plot(xxx, np.poly1d(myfit)(xxx), lw=3, label='Slope: {0:5.2g} +/- {1:5.2e}'.format(myfit[0], np.sqrt(covfit[0,0])))
legend()


The slope is the same as in the Full-sky case. So Full/Partial sky does not seem to have an effect (which makes if this is due to truncation of the harmonic spectrum)

# Signal Only

In [ ]:
#### PYSM Stuff ####
# generate I,Q,U sky maps with PySM
unit = u.uK_CMB
np.random.seed(42)
sky_cmb = pysm3.Sky(nside=nside, preset_strings=['d1'], output_unit=unit)
sky_emissions_cmb = sky_cmb.get_emission(150 * u.GHz)
maps_cmb = np.array([emission.value for emission in sky_emissions_cmb])


## 3) Full Sky - Signal Only

In [ ]:
nside = 256

# input map
mymap = maps_cmb[0,:]/np.std(maps_cmb[0,:])

# Anafast
cl, alm = hp.anafast(mymap, alm=True)

# Synfast
mapback = hp.alm2map(alm, nside)

# Maps
hp.mollview(mymap, sub=(1,3,1), min=np.min(mymap), max=np.max(mymap)/10, title='In')
hp.mollview(mapback, sub=(1,3,2), min=np.min(mymap), max=np.max(mymap)/10, title='Out')
hp.mollview(mapback-mymap, sub=(1,3,3), title='Out - In')

# Scatter Plot
figure()
plot(mymap, mapback, 'o')
xlabel('In')
ylabel('Out')
myfit, covfit = np.polyfit(mymap, mapback, 1, cov=True)
xxx = np.linspace(np.min(mymap),np.max(mymap), 100)
plot(xxx, xxx, 'r--', label='y=x')
plot(xxx, np.poly1d(myfit)(xxx), lw=3, label='Slope: {0:5.2g} +/- {1:5.2e}'.format(myfit[0], np.sqrt(covfit[0,0])))
legend()


## 4) Partial Sky - Signal Only

In [ ]:
nside = 256
covmap = hp.ud_grade(hp.read_map('QUBIC-Typical-Coverage.fits'), nside)
thr = 0.1
mymask = (covmap > (np.max(covmap)*thr)).astype(int)
pixok = mymask > 0

# input map
mymap = maps_cmb[0,:]/np.std(maps_cmb[0,:])
mymap[~pixok] = hp.UNSEEN


# Anafast
cl, alm = hp.anafast(mymap, alm=True)

# Synfast
mapback[~pixok] = hp.UNSEEN

diffmap = mapback-mymap
diffmap[~pixok] = hp.UNSEEN

# Maps
hp.mollview(mymap, sub=(1,3,1), min=np.min(mymap[pixok]), max=np.max(mymap[pixok])/10, title='In')
hp.mollview(mapback, sub=(1,3,2), min=np.min(mymap[pixok]), max=np.max(mymap[pixok])/10, title='Out')
hp.mollview(diffmap, sub=(1,3,3), title='Out - In')

# Scatter Plot
figure()
plot(mymap[pixok], mapback[pixok], 'o')
xlabel('In')
ylabel('Out')
myfit, covfit = np.polyfit(mymap[pixok], mapback[pixok], 1, cov=True)
xxx = np.linspace(np.min(mymap[pixok]),np.max(mymap[pixok]), 100)
plot(xxx, xxx, 'r--', label='y=x')
plot(xxx, np.poly1d(myfit)(xxx), lw=3, label='Slope: {0:5.2g} +/- {1:5.2e}'.format(myfit[0], np.sqrt(covfit[0,0])))
legend()


# Test the alm functions...

In [ ]:
# this function is from Simon Biquard
def get_alm_maps(pixel_maps, fwhms, resol_correction=False, ref_fwhm=0, pixwin_correction=False, verbose=False):
    """
    Compute alm maps from pixel maps and format them for FgBuster.
    """
    sh = np.shape(pixel_maps)
    nside = hp.npix2nside(sh[2])
    n = sh[0]
    lmax = 2*nside+1
    ell = np.arange(start=0, stop= lmax+1)

    ref_sigma_rad = np.deg2rad(ref_fwhm) / 2.355
    #ref_fl = np.exp(- 0.5 * np.square(ref_sigma_rad * ell))
    ref_fl = hp.gauss_beam(np.deg2rad(ref_fwhm), lmax=lmax)
    
    if verbose: 
        print('In get_alm_maps: FWHM = ', fwhms)
    beam_sigmas_rad = np.deg2rad(fwhms) / (2*np.sqrt(2*np.log(2)))
    pixwin = hp.pixwin(nside, lmax=lmax) if pixwin_correction else np.ones(lmax + 1)

    # compute maps
    #figure()
    alm_maps = None
    for f in range(n):
        alms = hp.map2alm(pixel_maps[f], lmax=lmax, pol=True)
        correction = None
        if f == 0:
            sh = np.shape(alms)
            alm_maps = np.empty((n, sh[0], 2 * sh[1]))
        if resol_correction:
            print('Applying Resol Correction')
            #gauss_fl = np.exp(- 0.5 * np.square(beam_sigmas_rad[f] * ell))
            gauss_fl = hp.gauss_beam(np.deg2rad(fwhms[f]), lmax=lmax)
            correction = ref_fl / gauss_fl / pixwin
            #plot(correction, label='freq {}'.format(f))
        else:
            print('No Resol Correction applied')
        for i, t in enumerate(alms):
            alm_maps[f, i] = format_alms(hp.almxfl(t, correction) if resol_correction else t)
    #legend()
    #title('Bl ratio in get_alm_maps')
    return alm_maps

# credits to J. Errard for these two functions
def intersect_mask(maps):
    if hp.pixelfunc.is_ma(maps):
        mask = maps.mask
    else:
        mask = maps == hp.UNSEEN

    # Mask entire pixel if any of the frequencies in the pixel is masked
    return np.any(mask, axis=tuple(range(maps.ndim - 1)))

def format_alms(alms, lmin=0, nulling_option=True):
    lmax = hp.Alm.getlmax(alms.shape[-1])
    alms = np.asarray(alms, order='C')
    alms = alms.view(np.float64)
    em = hp.Alm.getlm(lmax)[1]
    em = np.stack((em, em), axis=-1).reshape(-1)
    mask_em = [m != 0 for m in em]
    #alms[..., mask_em] *= np.sqrt(2)
    if nulling_option:
        alms[..., np.arange(1, lmax + 1, 2)] = hp.UNSEEN  # mask imaginary m = 0
        mask_alms = intersect_mask(alms)
        alms[..., mask_alms] = 0  # thus no contribution to the spectral likelihood
    alms = np.swapaxes(alms, 0, -1)
    if lmin != 0:
        ell = hp.Alm.getlm(lmax)[0]
        ell = np.stack((ell, ell), axis=-1).reshape(-1)
        mask_lmin = [ll < lmin for ll in ell]
        if nulling_option:
            alms[mask_lmin, ...] = hp.UNSEEN
    return alms


# Test:
on va prendre les fonctions ci-dessus (de FBG-CUtSky-NoNoise-WithFunctions) et voir si l'effet que l'on constate avec FBG peut provenir de ces fonctions et de la manière dont les alm sont ordonnés.

In [ ]:
nside = 256
covmap = hp.ud_grade(hp.read_map('QUBIC-Typical-Coverage.fits'), nside)
thr = 0.1
mymask = (covmap > (np.max(covmap)*thr)).astype(int)
pixok = mymask > 0


mymaps = maps_cmb.copy()
mymaps[:,~pixok] = hp.UNSEEN

### This is the standard alm proceedure
# Anafast
cl, alm = hp.anafast(mymaps, alm=True, lmax=2*nside+1)
# Synfast
backmaps_hp = hp.alm2map(alm, nside)
backmaps_hp[:,~pixok] = hp.UNSEEN



### This is using the functions in FGB-CutSky-NoNoise-WithFunctions
fwhms = 0
alms = get_alm_maps(np.reshape(mymaps, (1,3,12*nside**2)), fwhms, 
                    resol_correction=False, ref_fwhm=0, pixwin_correction=False, verbose=False)
backmaps = hp.alm2map(alms[0,:, ::2] + alms[0,:, 1::2]*1j, nside)
backmaps[:,~pixok] = hp.UNSEEN

for i in range(3):
    hp.gnomview(backmaps[i,:]-mymaps[i,:], sub=(1,2,1), rot=center, reso=20)
    hp.gnomview(backmaps_hp[i,:]-mymaps[i,:], sub=(1,2,2), rot=center, reso=15)

    figure()
for i in range(3):
    subplot(1,3,i+1)
    plot(mymaps[i,pixok], backmaps_hp[i,pixok], 'o', label='ALM function from Healpy')
    
    plot(mymaps[i,pixok], backmaps[i,pixok], 'o', label='ALM function from FGB')
    mini = np.min(mymaps[i,pixok])
    maxi = np.max(mymaps[i,pixok])
    xlim(mini, maxi)
    ylim(mini, maxi)
    x = np.linspace(mini, maxi)
    plot(x,x,'k--', label='y=x')
    myfit, covfit = np.polyfit(mymaps[i,pixok], backmaps[i,pixok], 1, cov=True)
    plot(x, np.poly1d(myfit)(x), lw=3, label='Slope: {0:5.2g} +/- {1:5.2e}'.format(myfit[0], np.sqrt(covfit[0,0])))
    legend()

In [ ]:
### healpy are alm
print('Healpy')
print(alm.dtype)
print(alm.shape)
print(alm[0,:20])

### method used in FGB functions
print('')
print('FGB')
new_alms = alms[0,:, ::2] + alms[0,:, 1::2]*1j
print(new_alms.dtype)
print(new_alms.shape)
print(new_alms[0,:20])

diff = new_alms[0,:]-alm[0,:]

for i in range(600):
    print('{}: {} - {} = {}'.format(i,alm[0,i], new_alms[0,i], diff[i]))
